## Circle STARK: FRI Prover and Verifier

This notebook demonstrates a simplified FRI (Fast Reed-Solomon Interactive Oracle Proofs of Proximity) proof system—one of the core components in Circle STARK protocols. 
- Commit Phase: Iteratively “fold” a vector of polynomial evaluations using random challenge values.
- Query Phase: Generate query proofs by extracting opening pairs from each folding round.
- Verification: Reconstruct the folded value from the query proofs and check that it matches the final constant.

The simulation uses simplified arithmetic and randomness to illustrate the main ideas behind the FRI prover and verifier.

In [1]:
import numpy as np
import random

def fold_evals(evals, beta):
    """
    Fold the evaluation vector by combining adjacent pairs with the challenge beta.
    
    Args:
        evals (np.array): The current evaluation vector (must have even length).
        beta (float): A randomly sampled challenge.
    
    Returns:
        np.array: A new evaluation vector of half the length.
    """
    new_evals = []
    for i in range(0, len(evals), 2):
        new_val = evals[i] + beta * evals[i+1]
        new_evals.append(new_val)
    return np.array(new_evals)

### Commit Phase (FRI Prover)

In the commit phase, the prover repeatedly folds the evaluation vector until its length reaches a predefined “blowup” factor (here, we use 1 for simplicity). Each folding round uses a fresh challenge value.

In [2]:
def commit_phase(evals, blowup=1):
    """
    Simulate the commit phase of FRI by iteratively folding the evaluation vector.
    
    Args:
        evals (np.array): The initial evaluation vector.
        blowup (int): The target length (e.g. 1).
    
    Returns:
        (list, final_poly): A tuple where the first element is a list of round data 
                              (each round containing the evaluation vector and beta)
                              and final_poly is the folded constant.
    """
    rounds = []
    current = evals
    while len(current) > blowup:
        beta = random.uniform(0.1, 2.0)  # simulate a random challenge
        round_data = {'evals': current, 'beta': beta}
        rounds.append(round_data)
        current = fold_evals(current, beta)
    final_poly = current[0]
    return rounds, final_poly

### Query Phase

For each query, the prover extracts “openings” from each commit round. In this simplified version, we assume that queries target even indices so that the corresponding pair in each round is well defined.

In [3]:
def answer_query(commit_rounds, query_index):
    """
    For a given (even) query index, extract the corresponding pair of values 
    (v0, v1) from each commit round.
    
    Args:
        commit_rounds (list): List of commit round data from the commit phase.
        query_index (int): The chosen query index (assumed even).
    
    Returns:
        list: A list of tuples (v0, v1) for each round.
    """
    proof = []
    current_index = query_index
    for round_data in commit_rounds:
        evals = round_data['evals']
        # Ensure current_index is even and in range
        q = current_index - (current_index % 2)
        if q+1 >= len(evals):
            q = len(evals) - 2  # adjust if out-of-bound
        v0 = evals[q]
        v1 = evals[q+1]
        proof.append((v0, v1))
        # For the next round, simulate the index update (integer division by 2)
        current_index //= 2
    return proof

### Prover Function

The prove function combines the commit phase and query generation. It returns a proof object containing:
- The commit rounds (with challenges and intermediate evaluation vectors)
- The final folded value (`final_poly`)
- A list of query proofs (each including the query index and opening pairs)
- A dummy proof-of-work witness

In [4]:
def prove(evals, num_queries=2, blowup=1):
    """
    Simulate the FRI prover by executing the commit phase and generating query proofs.
    
    Args:
        evals (np.array): The initial evaluation vector.
        num_queries (int): Number of queries to generate.
        blowup (int): Target length for the commit phase (e.g., 1).
    
    Returns:
        dict: A proof object containing commit rounds, final_poly, query proofs, and a PoW witness.
    """
    commit_rounds, final_poly = commit_phase(evals, blowup=blowup)
    query_proofs = []
    max_index = len(commit_rounds[0]['evals'])
    # For simplicity, choose random even indices within the range.
    for _ in range(num_queries):
        query_index = random.randrange(0, max_index, 2)
        proof_steps = answer_query(commit_rounds, query_index)
        query_proofs.append({'query_index': query_index, 'proof_steps': proof_steps})
    proof = {
        'commit_rounds': commit_rounds,
        'final_poly': final_poly,
        'query_proofs': query_proofs,
        'pow_witness': random.getrandbits(64)  # Dummy proof-of-work witness
    }
    return proof

### Verification Functions

The verifier uses the query proofs and commit rounds to re-fold the corresponding values and checks that the final result matches the claimed `final_poly`.

In [5]:
def verify_query(commit_rounds, query_proof, query_index, final_poly):
    """
    Verify a single query proof by re-folding the opened pairs.
    
    Args:
        commit_rounds (list): The commit rounds data.
        query_proof (list): The list of opening pairs for this query.
        query_index (int): The original query index.
        final_poly (complex): The claimed final folded value.
    
    Returns:
        bool: True if the recomputed value matches final_poly, False otherwise.
    """
    computed = None
    current_index = query_index
    for round_data, (v0, v1) in zip(commit_rounds, query_proof):
        beta = round_data['beta']
        folded = v0 + beta * v1
        computed = folded  # Update computed value for the round
        current_index //= 2  # Simulate index update
    return np.isclose(computed, final_poly)

def verify(proof, num_queries=2, blowup=1):
    """
    Simulate the FRI verifier by checking all query proofs.
    
    Args:
        proof (dict): The proof object produced by the prover.
        num_queries (int): Number of queries expected.
        blowup (int): Target length for the commit phase.
    
    Returns:
        bool: True if all query proofs verify, False otherwise.
    """
    commit_rounds = proof['commit_rounds']
    final_poly = proof['final_poly']
    for qp in proof['query_proofs']:
        query_index = qp['query_index']
        proof_steps = qp['proof_steps']
        if not verify_query(commit_rounds, proof_steps, query_index, final_poly):
            return False
    # Dummy PoW check (always passes in this simulation)
    return True

### Testing the Proof System

The following cell generates a random evaluation vector, runs the prover to produce a proof, and then verifies that proof.

In [6]:
# Set a random seed for reproducibility
np.random.seed(42)

# Generate a random evaluation vector of length 16 (must be a power of 2)
evals = np.random.random(16) + 1j * np.random.random(16)
print("Initial Evaluation Vector:")
print(evals)

# Run the FRI prover to generate a proof (using 3 queries for demonstration)
proof = prove(evals, num_queries=3, blowup=1)
print("\nGenerated Proof:")
print(proof)

# Verify the generated proof
is_valid = verify(proof, num_queries=3, blowup=1)
print("\nProof Verification Result:", is_valid)

Initial Evaluation Vector:
[0.37454012+0.30424224j 0.95071431+0.52475643j 0.73199394+0.43194502j
 0.59865848+0.29122914j 0.15601864+0.61185289j 0.15599452+0.13949386j
 0.05808361+0.29214465j 0.86617615+0.36636184j 0.60111501+0.45606998j
 0.70807258+0.78517596j 0.02058449+0.19967378j 0.96990985+0.51423444j
 0.83244264+0.59241457j 0.21233911+0.04645041j 0.18182497+0.60754485j
 0.18340451+0.17052412j]

Generated Proof:
{'commit_rounds': [{'evals': array([0.37454012+0.30424224j, 0.95071431+0.52475643j,
       0.73199394+0.43194502j, 0.59865848+0.29122914j,
       0.15601864+0.61185289j, 0.15599452+0.13949386j,
       0.05808361+0.29214465j, 0.86617615+0.36636184j,
       0.60111501+0.45606998j, 0.70807258+0.78517596j,
       0.02058449+0.19967378j, 0.96990985+0.51423444j,
       0.83244264+0.59241457j, 0.21233911+0.04645041j,
       0.18182497+0.60754485j, 0.18340451+0.17052412j]), 'beta': 1.698999847555588}, {'evals': array([1.98980358+1.19580334j, 1.74911462+0.92674328j,
       0.4210533

### Conclusion

In this notebook, we simulated a simplified version of the FRI proof system as used in Circle STARK protocols. We demonstrated:
- Commit Phase: Folding a polynomial evaluation vector using random challenge values.
- Query Phase: Extracting opening proofs corresponding to specific query indices.
- Verification: Recomputing folded values to ensure consistency with the final claimed constant.

This simulation—while simplified—captures the core idea of how FRI proofs ensure that a function is close to a low-degree polynomial, a key step in STARK-based proofs.